In [ ]:
import pandas as pd


file_list = [
    
    '/Users/rainfow/Desktop/Summer projects/companies_data/datetime_1_7.csv',
    '/Users/rainfow/Desktop/Summer projects/companies_data/datetime_2_7.csv',
    '/Users/rainfow/Desktop/Summer projects/companies_data/datetime_3_7.csv',
    '/Users/rainfow/Desktop/Summer projects/companies_data/datetime_4_7.csv',
    '/Users/rainfow/Desktop/Summer projects/companies_data/datetime_5_7.csv',
    '/Users/rainfow/Desktop/Summer projects/companies_data/datetime_6_7.csv',
    '/Users/rainfow/Desktop/Summer projects/companies_data/datetime_7_7.csv'
]

# 从CSV文件读取数据，并将日期列转换为datetime类型
df_list = [pd.read_csv(f, parse_dates=["IncorporationDate"]) for f in file_list]

# 合并所有数据框为一个大的数据框
df = pd.concat(df_list, ignore_index=True)
df.dropna(subset=['lat', 'long'], inplace=True)

In [ ]:
def filter_by_postcode_prefix(df, prefix):
    # Split the postcode at the space to get the prefix
    df['PostCodePrefix'] = df['RegAddress.PostCode'].str.split().str[0]
    # Filter the DataFrame based on the prefix
    df_filtered = df[df['PostCodePrefix'].str.startswith(prefix)]
    return df_filtered

df_filtered = filter_by_postcode_prefix(df, 'WC2')
df_filtered.head()

In [ ]:
import numpy as np

#df_filtered['Date'] = df_filtered['IncorporationDate'].dt.to_period('D')
df_filtered['Date'] = pd.to_datetime(df_filtered['IncorporationDate'])


##df_filtered_count = df_filtered.groupby(['RegAddress.PostCode', 'Date']).size().reset_index(name='CompanyCount')
##print(df_filtered_count.head(50))

df_filtered_count = df_filtered.groupby(['RegAddress.PostCode', 'Date']).agg({
    'CompanyName': lambda x: list(x),
    'lat': 'first',
    'long': 'first',
    'SICCode.SicText_1': lambda x: list(x),
}).reset_index()

# df_filtered_count['CompanyCount'] = df_filtered.groupby(['RegAddress.PostCode', 'Date']).size().values

df_filtered_count.head()


In [ ]:
import re

def check_CompanyName(df):
    # Define the scoring function for company names
    def score_company_name(company_name_list):
        for company_name in company_name_list:
            # Check conditions
            # 1. Starts with a digit
            #if re.match(r'^\d', company_name):
            #   return 1
            # 2. Starts with consecutive digits
            if re.match(r'^\d{3,}', company_name):
                return 1.5
            # 3. Starts with a special character (!, @, #, etc.)
            if re.match(r'^[!@#\$%\^&\*\(\)_\+\-=\[\]\{\};:\'",<>\?]', company_name):
                return 3
            # 4. Starts with three or more consecutive identical letters
            if re.match(r'^(.)\1{2,}', company_name):
                return 3
            # 5.Name length is less than 8 and starts with a number followed by a letter
            if len(company_name) < 8 or re.match(r'^\d+[A-Za-z]', company_name):
                return 3
        return 0
    
    # Calculate score for each row
    df['name_score'] = df['CompanyName'].apply(score_company_name)
    
    return df


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def integrated_similarity_scores(df):
    
    # 1. First function: compute_similarity_scores
    def compute_string_similarity(companies_1, companies_2):
        combined_companies = companies_1 + companies_2
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(combined_companies)
        cosine_similarities = cosine_similarity(tfidf_matrix)
        total_similarity = 0
        count = 0
        for i in range(len(companies_1)):
            for j in range(len(companies_1), len(combined_companies)):
                total_similarity += cosine_similarities[i][j]
                count += 1
        return total_similarity / count if count != 0 else 0

    def compute_name_score(group):
        scores = [0] * len(group)
        if len(group) >= 2:
            # 找出间隔最短的两个日期的索引
            min_diff = float('inf')
            idx1, idx2 = None, None
            sorted_dates = sorted(range(len(group)), key=lambda x: group.iloc[x]['Date'])
            
            for i in range(len(sorted_dates) - 1):
                date1 = pd.to_datetime(group.iloc[sorted_dates[i]]['Date'])
                date2 = pd.to_datetime(group.iloc[sorted_dates[i + 1]]['Date'])
                date_diff = (date2 - date1).days
                if date_diff < min_diff:
                    min_diff = date_diff
                    idx1, idx2 = sorted_dates[i], sorted_dates[i + 1]

            # 获取间隔最短的两个日期的公司名称列表
            recent_companies_1 = group.iloc[idx1]['CompanyName']
            recent_companies_2 = group.iloc[idx2]['CompanyName']
            if isinstance(recent_companies_1, str):
                recent_companies_1 = [recent_companies_1]
            if isinstance(recent_companies_2, str):
                recent_companies_2 = [recent_companies_2]
            # 计算公司名称的平均相似性
            avg_similarity = compute_string_similarity(recent_companies_1, recent_companies_2)
            threshold = 0.5
            if avg_similarity > threshold:
                scores[idx1] = 1
                scores[idx2] = 1
        return scores
    
    # 2. Second function: compute_siccode_similarity_scores
    def compute_siccode_score(group):
        scores = [0] * len(group)
        if len(group) >= 2:
            name_scores = compute_name_score(group)
            for i in range(len(name_scores)):
                
                # Only consider rows with similarity score of 1
                if name_scores[i] == 1:  
                    if i + 1 < len(name_scores) and name_scores[i + 1] == 1:
                        if group.iloc[i]['SICCode.SicText_1'] == group.iloc[i + 1]['SICCode.SicText_1']:
                            scores[i] = 1
                            scores[i + 1] = 1
        return scores
    
    # Applying functions and adding new columns to df
    df['name_similarity_score'] = [score for sublist in df.groupby('RegAddress.PostCode').apply(compute_name_score).reset_index(drop=True) for score in sublist]
    df['siccode_similarity_score'] = [score for sublist in df.groupby('RegAddress.PostCode').apply(compute_siccode_score).reset_index(drop=True) for score in sublist]
    
    return df


In [ ]:
# Combine both scoring methods and then aggregate for the final result
def compute_combined_scores(df):
    # Get the scores from both methods
    df_name_scores = check_CompanyName(df)
    df_similarity_scores = integrated_similarity_scores(df)
    
    # Combine both scores
    df['name_score'] = df_name_scores['name_score']
    df['name_similarity_score'] = df_similarity_scores['name_similarity_score']
    df['siccode_similarity_score'] = df_similarity_scores['siccode_similarity_score']

    
    # Calculate the total score
    df['Total_Score'] = df['name_score'] + df['name_similarity_score']+df['siccode_similarity_score']
    
    return df

# Compute the combined scores
df_text_scores = compute_combined_scores(df_filtered_count)


In [ ]:
# Group by postcode to get the total score for each postcode
similarity_scores_output = df_text_scores.groupby(['RegAddress.PostCode', 'lat', 'long']).agg({
    'CompanyName': 'size',
    'Total_Score': 'max'
}).reset_index()

similarity_scores_output.rename(columns={'CompanyName': 'CompanyCount'}, inplace=True)
similarity_scores_output.head(10)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Basic statistical description of Total_Score
score_description = similarity_scores_output["Total_Score"].describe()

# Plot the histogram for Total_Score
plt.figure(figsize=(10,6))
sns.histplot(similarity_scores_output["Total_Score"], kde=True, bins=30)
plt.title("Distribution of Total Scores")
plt.xlabel("Total Score")
plt.ylabel("Number of Postcodes")
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

score_description


In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx
import pandas as pd

# Create a new figure
fig, ax = plt.subplots(figsize=(10, 10))

# Define the color and size mapping based on the quantiles of 'SumScores'
def map_color_size(score):
    threshold_1 = 0
    threshold_2 = 1
    threshold_3 = 2

    if score <= threshold_1:
        return 'green', 5, 'Not very suspicious'
    elif score <= threshold_2:
        return 'blue', 10, 'Slightly suspicious'
    elif score <= threshold_3:
        return 'orange', 20, 'Fairly suspicious'
    else:
        return 'red', 100, 'Very suspicious'

similarity_scores_output['color'], similarity_scores_output['size'], similarity_scores_output['label'] = zip(*similarity_scores_output['Total_Score'].apply(map_color_size))

# Calculate the total number of postcodes
total_postcodes = len(similarity_scores_output)

# Define a dictionary for labels, colors, and sizes
label_dict = {
    'Not very suspicious': {'color': 'green', 'size': 5},
    'Slightly suspicious': {'color': 'blue', 'size': 10},
    'Fairly suspicious': {'color': 'orange', 'size': 20},
    'Very suspicious': {'color': 'red', 'size': 100}
}

# Scatter plot
for label, attr in label_dict.items():
    idx = similarity_scores_output['label'] == label
    postcode_count = len(similarity_scores_output.loc[idx])
    percentage = postcode_count / total_postcodes * 100
    ax.scatter(similarity_scores_output.loc[idx, 'long'], similarity_scores_output.loc[idx, 'lat'], c=attr['color'], s=attr['size'], label=f'{label} ({percentage:.1f}%)')


    # If the label is "Very suspicious" (which corresponds to red), add the postal code text
    if label == "Very suspicious":
        for x, y, postcode in zip(similarity_scores_output.loc[idx, 'long'], similarity_scores_output.loc[idx, 'lat'], similarity_scores_output.loc[idx, 'RegAddress.PostCode']):
            ax.text(x, y, postcode, ha='right', fontsize=7.5, color='black')

# Add legend
ax.legend(title="Suspicion Level")

# Add basemap
ctx.add_basemap(ax, crs='EPSG:4326', source=ctx.providers.Stamen.TonerLite)

# Show the plot
plt.show()
